In [2]:
# For ubuntu
import findspark
import os

findspark.init()

In [3]:
# All imports
import os
import urllib
import zipfile
import math
from pyspark.mllib.recommendation import ALS

In [5]:
# Denoting spark context
from pyspark import SparkContext
sc = SparkContext()

In [16]:
# Path
path_to_folder = os.path.join('/home/user/Desktop/datasets/')
reviews_binary = os.path.join(path_to_folder, 'BDA', 'In_Recommender_Format.csv')
reviews_binary_RDD = sc.textFile(reviews_binary)

In [17]:
# Taking the header line
reviews_binary_header = reviews_binary_RDD.take(1)[0]

In [18]:
small_ratings_data = reviews_binary_RDD.filter(lambda line: line!=reviews_binary_header).map(lambda line:
line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [19]:
small_ratings_data.take(3)

[(u'2', u'2', u'0'), (u'3', u'3', u'1'), (u'4', u'4', u'0')]

In [35]:
# Training, Validation, Testing
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [36]:
seed = 5L
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

In [37]:
min_error = float('inf')
best_rank = -1
best_iteration = -1

In [38]:
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations, lambda_=regularization_parameter)
    predictions = model.predictAll(validatma
                                   ion_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'For rank %s the RMSE is %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank

For rank 4 the RMSE is 0.560625660088
For rank 8 the RMSE is 0.465883123146
For rank 12 the RMSE is 0.461724852034


In [39]:
print 'The best model was trained with rank %s' % best_rank

The best model was trained with rank 12


In [40]:
predictions.take(3)

[((45832, 32676), 0.8830265827247685),
 ((35796, 11852), 0.14056994648632382),
 ((44636, 11852), 0.13200275580746024)]

In [41]:
rates_and_preds.take(3)

[((28478, 57452), (1.0, 0.11003208084243271)),
 ((56313, 86911), (1.0, 0.07989422484525688)),
 ((50347, 28233), (0.0, 0.036094304237628036))]

In [42]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations, lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)

For testing data the RMSE is 0.464306284121
